\pagenumbering{gobble}

\ 
\ 
\ 
\ 
\ 
\ 
\ 
\ 
\ 


In [40]:
# imports
import cobra
from libsbml import *
import refinegems as rg
import pandas as pd
import numpy as np
import tabulate

In [41]:
def find_additives(model, base_medium):
    with model:
        medium = model.medium
        model.medium = base_medium
        sol = model.optimize()      
    base_growth = sol.objective_value
    dt = (np.log(2) / sol.objective_value) * 60
    print('Growth rate without additives: ' + str(round(base_growth, 4)))
    print('Doubling time without additives: ' + str(round(dt, 2)) + ' min')
        
    enhancement = {}
    for ex in list(model.exchanges):
        if ex.id not in base_medium.keys():
            with model:
                medium = model.medium
                base_medium.update({ex.id:10.0})
                model.medium = base_medium
                sol = model.optimize()
                if sol.objective_value > base_growth:
                    enhancement[ex.id] = sol.objective_value - base_growth
                base_medium.pop(ex.id)
    
    enh = pd.DataFrame(enhancement.items(), columns=['exchange', 'diff']).sort_values(by=['diff'], ascending=False)
    
    return enh

In [42]:
def enhanced_growth(model, base_medium, new_metabs):
    local_medium = base_medium
    with model:
        medium = model.medium
        for met in new_metabs:
            local_medium[met] = 10.0
        model.medium = local_medium
        sol = model.optimize()
    base_growth = sol.objective_value
    dt = (np.log(2) / sol.objective_value) * 60
    print('Growth rate with additives: ' + str(round(base_growth, 4)))
    print('Doubling time with additives: ' + str(round(dt, 2)) + ' min')

In [43]:
CasA = rg.growth.load_all_media_from_db('../../refinegems/data/media_db.csv')[9]['BiGG_EX'].to_list()
CGXII = pd.read_csv('../analysis/CGXII_base_medium/CGXII.csv').set_index('exchange').to_dict()['flux']

## strain 16

In [44]:
model = rg.load.load_model_cobra('../models/Cstr_16.xml')
CGXII_comp_16 = pd.read_csv('../analysis/CGXII_base_medium/CGXII_comp_16.csv').set_index('exchange').to_dict()['flux']

In [45]:
for rea in model.metabolites.get_by_id('malt_c').reactions:
    print(rea)

MALTabc: atp_c + h2o_c + malt_e --> adp_c + h_c + malt_c + pi_c
AMALT1: malt_c + malttr_c --> glc__D_c + maltttr_c
MLTG1: h2o_c + malttr_c --> glc__D_c + malt_c
AMALT4: malt_c + malthx_c --> glc__D_c + malthp_c
AMALT2: malt_c + maltttr_c --> glc__D_c + maltpt_c
AMALT3: malt_c + maltpt_c --> glc__D_c + malthx_c
MTI: tre_c <=> malt_c


In [46]:
print(model.id)
enh = find_additives(model, CGXII_comp_16)
enh.to_csv('../analysis/additives/Cstr_16_adds.csv', index=False)
print(enh.head().to_markdown(index=False))

fda_1115
Growth rate without additives: 0.5579
Doubling time without additives: 74.54 min
| exchange    |     diff |
|:------------|---------:|
| EX_malthp_e | 2.03237  |
| EX_tre_e    | 0.478205 |
| EX_sucr_e   | 0.478205 |
| EX_malt_e   | 0.478205 |
| EX_lcts_e   | 0.418429 |


Going through the given list I found that L-Arginine and N-Acetyl-D-glucosamine are available in the lab storage. Simulation with both metabolites leads to a growth value of 0.8 which corresponds to a doubling time of 52 Minutes.

In [47]:
enhanced = ['EX_arg__L_e', 'EX_acgam_e']
enhanced_growth(model, CGXII_comp_16, enhanced)

Growth rate with additives: 0.8025
Doubling time with additives: 51.82 min


In [48]:
CGXII_base = CGXII_comp_16.copy()
for metab, flux in CGXII_comp_16.items():
    if metab not in CGXII.keys():
        CGXII_base.pop(metab)
CGXII_base

{'EX_nh4_e': 10.0,
 'EX_so4_e': 10.0,
 'EX_k_e': 10.0,
 'EX_pi_e': 10.0,
 'EX_urea_e': 10.0,
 'EX_ca2_e': 10.0,
 'EX_cl_e': 10.0,
 'EX_mg2_e': 10.0,
 'EX_fe2_e': 10.0,
 'EX_mn2_e': 10.0,
 'EX_cu2_e': 10.0,
 'EX_zn2_e': 10.0,
 'EX_glc__D_e': 10.0,
 'EX_h2o_e': 10.0,
 'EX_o2_e': 20.0,
 'EX_h_e': 10.0}

In [49]:
enhanced_growth(model, CGXII_base, CasA)

Growth rate with additives: 0.0
Doubling time with additives: 2750479021271774.5 min


## strain 15

In [50]:
model = rg.load.load_model_cobra('../models/Cstr_15.xml')
CGXII_comp_15 = pd.read_csv('../analysis/CGXII_base_medium/CGXII_comp_15.csv').set_index('exchange').to_dict()['flux']

In [51]:
print(model.id)
enh = find_additives(model, CGXII_comp_15)
enh.to_csv('../analysis/additives/Cstr_15_adds.csv', index=False)
print(enh.head().to_markdown(index=False))

fda_1197
Growth rate without additives: 0.5522
Doubling time without additives: 75.31 min
| exchange    |     diff |
|:------------|---------:|
| EX_malthp_e | 2.01175  |
| EX_malt_e   | 0.473353 |
| EX_tre_e    | 0.473353 |
| EX_sucr_e   | 0.473353 |
| EX_lcts_e   | 0.384599 |


Also shows enhancement by L-Arginine.

In [52]:
enhanced = ['EX_arg__L_e', 'EX_acgam_e']
enhanced_growth(model, CGXII_comp_15, enhanced)

Growth rate with additives: 0.5916
Doubling time with additives: 70.3 min


Lets test enhancement by Casamino acids.

In [53]:
enhanced_growth(model, CGXII_comp_15, CasA)

Growth rate with additives: 0.6439
Doubling time with additives: 64.59 min


In [54]:
CGXII_base = CGXII_comp_15.copy()
for metab, flux in CGXII_comp_15.items():
    if metab not in CGXII.keys():
        CGXII_base.pop(metab)
CGXII_base

{'EX_nh4_e': 10.0,
 'EX_so4_e': 10.0,
 'EX_k_e': 10.0,
 'EX_pi_e': 10.0,
 'EX_urea_e': 10.0,
 'EX_ca2_e': 10.0,
 'EX_cl_e': 10.0,
 'EX_mg2_e': 10.0,
 'EX_fe2_e': 10.0,
 'EX_mn2_e': 10.0,
 'EX_cu2_e': 10.0,
 'EX_zn2_e': 10.0,
 'EX_btn_e': 10.0,
 'EX_glc__D_e': 10.0,
 'EX_h2o_e': 10.0,
 'EX_o2_e': 20.0,
 'EX_h_e': 10.0}

In [55]:
enhanced_growth(model, CGXII_base, CasA)

Growth rate with additives: -0.0
Doubling time with additives: -1.3582553683821924e+16 min


## strain 14

In [56]:
model = rg.load.load_model_cobra('../models/Cstr_14.xml')
CGXII_comp_14 = pd.read_csv('../analysis/CGXII_base_medium/CGXII_comp_14.csv').set_index('exchange').to_dict()['flux']

In [57]:
print(model.id)
enh = find_additives(model, CGXII_comp_14)
enh.to_csv('../analysis/additives/Cstr_14_adds.csv', index=False)
print(enh.head().to_markdown(index=False))

fda_1054
Growth rate without additives: 0.5485
Doubling time without additives: 75.82 min
| exchange    |     diff |
|:------------|---------:|
| EX_malthp_e | 1.99828  |
| EX_sucr_e   | 0.470184 |
| EX_tre_e    | 0.470184 |
| EX_malt_e   | 0.470184 |
| EX_lcts_e   | 0.411411 |


L-Ascorbate is available in the lab.

In [58]:
enhanced = ['EX_ascb__L_e']
enhanced_growth(model, CGXII_comp_14, enhanced)

Growth rate with additives: 0.7445
Doubling time with additives: 55.86 min


In [59]:
enhanced_growth(model, CGXII_comp_14, CasA)

Growth rate with additives: 0.8624
Doubling time with additives: 48.23 min


In [60]:
CGXII_base = CGXII_comp_14.copy()
for metab, flux in CGXII_comp_14.items():
    if metab not in CGXII.keys():
        CGXII_base.pop(metab)
CGXII_base

{'EX_nh4_e': 10.0,
 'EX_so4_e': 10.0,
 'EX_k_e': 10.0,
 'EX_pi_e': 10.0,
 'EX_urea_e': 10.0,
 'EX_ca2_e': 10.0,
 'EX_cl_e': 10.0,
 'EX_mg2_e': 10.0,
 'EX_fe2_e': 10.0,
 'EX_mn2_e': 10.0,
 'EX_cu2_e': 10.0,
 'EX_zn2_e': 10.0,
 'EX_btn_e': 10.0,
 'EX_glc__D_e': 10.0,
 'EX_h2o_e': 10.0,
 'EX_o2_e': 20.0,
 'EX_h_e': 10.0}

In [61]:
enhanced_growth(model, CGXII_base, CasA)

Growth rate with additives: 0.0
Doubling time with additives: 8.554903579813985e+17 min


## strain 17

In [62]:
model = rg.load.load_model_cobra('../models/Cstr_17.xml')
CGXII_comp_17 = pd.read_csv('../analysis/CGXII_base_medium/CGXII_comp_17.csv').set_index('exchange').to_dict()['flux']

In [63]:
print(model.id)
enh = find_additives(model, CGXII_comp_17)
print(enh.head().to_markdown(index=False))

fda_1116
Growth rate without additives: 0.0
Doubling time without additives: inf min


/var/folders/s0/m197rw591kxcl13y56pqhf9m0000gs/T/ipykernel_53527/4155444981.py:7: RuntimeWarning: divide by zero encountered in double_scalars
  dt = (np.log(2) / sol.objective_value) * 60


| exchange    |     diff |
|:------------|---------:|
| EX_s_e      | 0.622009 |
| EX_cgly_e   | 0.613182 |
| EX_cys__L_e | 0.612059 |
| EX_h2s_e    | 0.611861 |
| EX_gthrd_e  | 0.610023 |


No simple additives found since there is not growth just on the CGXII medium even under the addition of `pnto_R`, `nmn` and `cobalt`. The next test is to try and find essential exchanges using refineGEMs and add those to the medium.

In [64]:
ess = rg.growth.get_essential_reactions_via_bounds(model)
for entry in ess:
    CGXII_comp_17.update({entry:10.0})

In [65]:
enh = find_additives(model, CGXII_comp_17)
print(enh.head().to_markdown(index=False))

Growth rate without additives: -0.0
Doubling time without additives: -1.8774713838470752e+30 min
| exchange    |     diff |
|:------------|---------:|
| EX_s_e      | 0.622009 |
| EX_cgly_e   | 0.613182 |
| EX_cys__L_e | 0.612059 |
| EX_h2s_e    | 0.611861 |
| EX_gthrd_e  | 0.610023 |


Still no reliable growth value. Next test is to add Casamino acids using the media definition given in refineGEMs.

In [66]:
enhanced_growth(model, CGXII_comp_17, CasA)

Growth rate with additives: 0.7301
Doubling time with additives: 56.96 min


In [67]:
CGXII_comp_17 = pd.read_csv('../analysis/CGXII_base_medium/CGXII_comp_17.csv').set_index('exchange').to_dict()['flux']
for ex in CasA:
    CGXII_comp_17.update({ex:10.0})

In [68]:
enh = find_additives(model, CGXII_comp_17)
enh.to_csv('../analysis/additives/Cstr_17_adds.csv', index=False)
print(enh.head().to_markdown(index=False))

Growth rate without additives: 0.7301
Doubling time without additives: 56.96 min
| exchange    |     diff |
|:------------|---------:|
| EX_malthp_e | 2.25677  |
| EX_raffin_e | 0.796507 |
| EX_malt_e   | 0.531005 |
| EX_tre_e    | 0.531005 |
| EX_sucr_e   | 0.531005 |


Strain 17 should be able to grow on CGXII under the addition of Casamino Acids.

In [69]:
enhanced_growth(model, CGXII_comp_17, CasA)

Growth rate with additives: 0.7301
Doubling time with additives: 56.96 min


In [70]:
CGXII_base = CGXII_comp_17.copy()
for metab, flux in CGXII_comp_17.items():
    if metab not in CGXII.keys():
        CGXII_base.pop(metab)
CGXII_base

{'EX_nh4_e': 10.0,
 'EX_so4_e': 10.0,
 'EX_k_e': 10.0,
 'EX_pi_e': 10.0,
 'EX_urea_e': 10.0,
 'EX_ca2_e': 10.0,
 'EX_cl_e': 10.0,
 'EX_mg2_e': 10.0,
 'EX_fe2_e': 10.0,
 'EX_mn2_e': 10.0,
 'EX_cu2_e': 10.0,
 'EX_zn2_e': 10.0,
 'EX_glc__D_e': 10.0,
 'EX_h2o_e': 10.0,
 'EX_o2_e': 20.0,
 'EX_h_e': 10.0}

In [71]:
enhanced_growth(model, CGXII_base, CasA)

Growth rate with additives: 0.0
Doubling time with additives: 14843514922900.7 min
